In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import math

In [ ]:
% cd /content/gdrive/'My Drive'/'NLP'/'Final Project'/data


# !cd drive/NLP/'Final Project'


/content/gdrive/My Drive/NLP/Final Project/data


In [ ]:
dataset = 'partial'

In [ ]:

def get_word_and_tag(path):
    try:
        words = []
        tags = []
        word_tag = []
        f = open(path + '/train',encoding = 'utf-8')
        data = f.readlines()
        for x in data:
            if len(x)<2:
#                 print(x)
                continue
            line = x.rstrip()
            temp = line.split(' ')
            word_tag.append(temp)
            words.append(temp[0])
            tags.append(temp[1])
        return words,tags,word_tag
    finally:
        f.close()
        

def get_dictionaries(word_tag):
    label_count = dict()
    for i in word_tag:
        label_count[i[1]] = label_count.get(i[1], 0) + 1
        

    word_count = dict()
    for i in word_tag:
        word_count[i[0]] = word_count.get(i[0], 0) + 1
    
    y_to_x = dict()
    for i in word_tag:
        y_to_x[i[1]+'+'+i[0]] = y_to_x.get(i[1]+'+'+i[0], 0) + 1
        
                  
    return label_count,word_count,y_to_x

In [ ]:
data_word ,data_tags, data_word_tag = get_word_and_tag(dataset)


In [ ]:
data_label_count, data_word_count , data_y_to_x = get_dictionaries(data_word_tag)

In [ ]:
print(data_word_tag)

[['The', 'O'], ['official', 'O'], ['cause', 'O'], ['of', 'O'], ['death', 'O'], ['has', 'O'], ['not', 'O'], ['been', 'O'], ['officially', 'O'], ['determined', 'O'], [',', 'O'], ['but', 'O'], ['investigators', 'O'], ['believe', 'O'], ['the', 'O'], ['36-year-old', 'O'], ['writer', 'O'], ['died', 'O'], ['from', 'O'], ['a', 'O'], ['self-inflicted', 'O'], ['gunshot', 'O'], ['wound', 'O'], ['.', 'O'], ['Mr.', 'B-per'], ['Omi', 'I-per'], ['called', 'O'], ['for', 'O'], ['stronger', 'O'], ['political', 'O'], ['will', 'O'], ['by', 'O'], ['Asian', 'O'], ['governments', 'O'], ['to', 'O'], ['stop', 'O'], ['the', 'O'], ['spread', 'O'], ['of', 'O'], ['the', 'O'], ['disease', 'O'], ['.', 'O'], ['This', 'O'], ['is', 'O'], ['the', 'O'], ['second', 'O'], ['U.N.-Congolese', 'O'], ['offensive', 'O'], ['against', 'O'], ['militias', 'O'], ['in', 'O'], ['the', 'O'], ['region', 'O'], ['since', 'O'], ['the', 'O'], ['DRC', 'B-geo'], ["'s", 'O'], ['constitutional', 'O'], ['referendum', 'O'], ['a', 'O'], ['week', '

### Emissions


In [ ]:
def get_emission(x,y,xy_dict,label_dict):
    if y+'+'+x not in xy_dict:
        return 0.0
    count_y_to_x = xy_dict[y+'+'+x]
    count_y = label_dict[y]
    return count_y_to_x/count_y

In [ ]:
def get_emission_dict(word_and_tag,y_to_x,y_counts):
  ret = dict()
  for i in word_and_tag:
    ret['emission:'+i[1]+'+'+i[0]] = math.log(get_emission(i[0],i[1],y_to_x,y_counts))
  return ret

In [ ]:
print(get_emission_dict(data_word_tag,data_y_to_x,data_label_count))

{'emission:O+The': -4.333515843240958, 'emission:O+official': -7.166729187297174, 'emission:O+cause': -8.776167099731275, 'emission:O+of': -3.518671727703493, 'emission:O+death': -7.166729187297174, 'emission:O+has': -4.649032714686183, 'emission:O+not': -5.97280671882474, 'emission:O+been': -5.755742213586912, 'emission:O+officially': -8.370701991623111, 'emission:O+determined': -8.370701991623111, 'emission:O+,': -3.3359162372945717, 'emission:O+but': -6.211217742269738, 'emission:O+investigators': -9.46931428029122, 'emission:O+believe': -9.46931428029122, 'emission:O+the': -2.8532490951584024, 'emission:O+36-year-old': -9.46931428029122, 'emission:O+writer': -9.46931428029122, 'emission:O+died': -7.389872738611384, 'emission:O+from': -5.192648161275165, 'emission:O+a': -3.7522865788849984, 'emission:O+self-inflicted': -9.46931428029122, 'emission:O+gunshot': -9.46931428029122, 'emission:O+wound': -9.46931428029122, 'emission:O+.': -2.922528869530696, 'emission:B-per+Mr.': -1.658228

In [ ]:
emission_dictionary  = get_emission_dict(data_word_tag,data_y_to_x,data_label_count)

### Transitions

In [ ]:
def get_y_to_y(path):   
    try:
        f = open(path,encoding = 'utf-8')
        try_y = f.readlines()
    finally:
        f.close()
   
    sentences = []
    sentences.append('START')
    for line in try_y:
        x = line.split(' ')
    #     print(x)
        if len(x) <2:
            # linebreak
            sentences.append('STOP')
            sentences.append('START')
        else:
            sentences.append(x)
    states = []
    
    for term in range(len(sentences)-1): ## removes the last start
        if len(sentences[term])==2:
            states.append(sentences[term][1])
        else:
            states.append(sentences[term])
    for state in range(len(states)):
        states[state] = states[state].rstrip()
    return states
 



def get_y_to_y_count(states):
    y_to_y = dict()
    for i in range(len(states)-1):
        if(states[i+1]=='START'):
          continue
        y_to_y['transition:'+states[i]+ '+' +states[i+1]] = y_to_y.get('transition:'+states[i]+ '+'+states[i+1], 0) + 1 
        
    label_count_with_ss = dict()
    for i in range(len(states)):
        label_count_with_ss[states[i]] = label_count_with_ss.get(states[i], 0) + 1 
    return y_to_y , label_count_with_ss



In [ ]:
data_states = get_y_to_y(dataset + "/train")
print(data_states)

['START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'STOP', 'START', 'B-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'STOP', 'START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'STOP', 'START', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'O', 'B-per', 'I-per', 'I-per', 'I-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'STOP', 'START', 'B-gpe', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'STOP', 'START', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'STOP', 'START', 'B-gpe', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'I-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'STOP', 'START', 'O', 'O'

In [ ]:
y_to_y_counts , labels_with_start_stop= get_y_to_y_count(data_states)

In [ ]:
def get_transmission(u,v,yy_dict,label_dict):
    # u is i , v is i+1
    if 'transition:'+u+ '+' +v not in yy_dict:
        return 0.0
    count_u_to_v = yy_dict['transition:'+u+ '+' +v]
    count_y = label_dict[u]
    result = count_u_to_v/count_y
    return result


In [ ]:
def get_transmission_dict(emission_dict,y_to_y,y_count,states):
    
    for i in range(len(states)-1):
      if(states[i+1]=='START'):
        continue
      emission_dict['transition:'+states[i]+ '+' +states[i+1]] = math.log(get_transmission(states[i],states[i+1],y_to_y,y_count))
    return emission_dict


  



In [ ]:
resulting_dict = get_transmission_dict(emission_dictionary,y_to_y_counts,labels_with_start_stop,data_states)

In [ ]:
print(resulting_dict['transition:B-geo+I-geo'])

-1.5945122622174248
